In [ ]:
import numpy as np
from zipfile import ZipFile
import re
import torch
from torch import nn, optim
from sklearn.model_selection import train_test_split
#from google.colab import drive
#drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Import data

!wget -O x.zip "https://drive.google.com/uc?export=download&id=1221858Ognjuffo3Pv6QezSiBw8xBOBe-"

with ZipFile('/content/x.zip', 'r') as f:
  f.extract('x.txt')

x_real = []
x_imag = []
x = []
#with open('drive/MyDrive/x.txt') as input_file:
with open('x.txt') as input_file:
    for line in input_file:
      temp = re.sub(r'[()]', "", line)
      temp = complex(temp)
      x.append((temp.real, temp.imag))
      x_real.append(temp.real)
      x_imag.append(temp.imag)

y_real = []
y_imag = []
#with open('drive/MyDrive/y.txt') as input_file:
with open('y.txt') as input_file:
    for line in input_file:
      temp = re.sub(r'[()]', "", line)
      temp = complex(temp)
      y_real.append(temp.real)
      y_imag.append(temp.imag)

In [ ]:
# Conversion to TensorFlow

x_real_t = torch.Tensor(x_real).reshape(-1,1)
x_imag_t = torch.Tensor(x_imag).reshape(-1,1)
X = torch.stack((x_real_t, x_imag_t), -1)

y_real_t = torch.Tensor(y_real).reshape(-1,1)
y_imag_t = torch.Tensor(y_imag).reshape(-1,1)
Y = torch.stack((y_real_t, y_imag_t), -1)

In [ ]:
# Divide your data into training, validation and test dataset

x_train, x_test =  train_test_split(X, test_size=0.2, shuffle=False)
y_train, y_test =  train_test_split(Y, test_size=0.2, shuffle=False)

x_val, x_test =  train_test_split(X, test_size=0.5, shuffle=False)
y_val, y_test =  train_test_split(Y, test_size=0.5, shuffle=False)

In [ ]:
# Define nonlinear NN for equalizer

class NonlinEq(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin1 = nn.Linear(2, 50)
        self.relu1 = nn.ReLU()
        self.lin2 = nn.Linear(50, 20)
        self.relu2 = nn.ReLU()
        self.lin3 = nn.Linear(20,2)

    def forward(self, y):
        y = self.relu1(self.lin1(y))
        y = self.relu2(self.lin2(y))
        return self.lin3(y)[::2]

In [ ]:
# Define loss function

loss_fn = nn.MSELoss()
lr = 1e-2  # learning rate
num_epochs = 101

In [ ]:
# Initialize NN

torch.manual_seed(5)  
nleq = NonlinEq()
optimizer = optim.Adam(nleq.parameters(), lr=lr)

loss_last_batch = []
loss_val_set = []

In [ ]:
# Trainings loop

for j in range(num_epochs):
    y_eq = nleq(Y)
    loss = loss_fn(y_eq, X)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    # store loss of the last batch 
    loss_last_batch.append(loss.detach().numpy())
    # store loss of validation set 
    loss_val = loss_fn(nleq(y_val), x_val)
    loss_val_set.append(loss_val.detach().numpy())

    if j % 10 == 0:
        print(f'epoch {j}: Loss = {loss.detach().numpy() :.4f}')

epoch 0: Loss = 0.5115
epoch 10: Loss = 0.0752
epoch 20: Loss = 0.0364
epoch 30: Loss = 0.0259
epoch 40: Loss = 0.0209
epoch 50: Loss = 0.0176
epoch 60: Loss = 0.0156
epoch 70: Loss = 0.0146
epoch 80: Loss = 0.0138
epoch 90: Loss = 0.0130
epoch 100: Loss = 0.0124


In [ ]:
# Save result of the NN

torch.save(loss_last_batch, 'last_batch.pt')
torch.save(loss_val_set, 'val.pt')
torch.save(nleq, 'equal.pth')